In [117]:
import numpy as np
import pandas as pd
from numpy.linalg import eig
import matplotlib.pyplot as plt
from scipy.special import binom
from scipy.stats import norm

In [119]:
norm.pdf(0)

0.3989422804014327

In [128]:
a = 2
type(2*2)

int

In [ ]:
def eval_posterior_real(Y_r: np.ndarray, As_r_swap: np.ndarray, y_hat: np.ndarray, lambda_: float, theta: float):
    s1, s2 = Y_r.shape
    d, r = int(s1 / 2), int(s2 / 2)
    Y_rho_r_outer = Y_r @ np.conj(Y_r.T)
    y = np.trace(As_r_swap @ Y_rho_r_outer, axis1=1, axis2=2)

    lik = np.exp(
        - lambda_ * np.linalg.norm(y_hat - np.sqrt(2) * y) ** 2
    )
    prior = np.linalg.det(theta**2 * np.eye(2 * d) / np.sqrt(2)
                + np.sqrt(2) * Y_rho_r_outer) ** ((2*d + r + 2)/4)
    return lik * prior

In [123]:
def eval_proposal(Y: np.ndarray, dist: str = "normal"):
    # TODO
    m, n = Y.shape
    if dist == "normal":
        beta = 1
        return 1/(2 * np.pi )**(m*n*beta/2) * np.exp(-1/2 * np.linalg.norm(Y, ord="fro")**2)
    elif dist == "goe":
        assert m == n, "d and r must have the same value for GOE"
        Z = (4 * np.pi)**(m/2) * (2*np.pi)**(1/2 * binom(m, 2))
        return np.exp(-1/4 * np.trace(Y @ Y)) / Z
def sample_proposal(d: int, r: int, dist: str):
    if dist == "normal":
        return np.random.randn(d, r)
    elif dist == "goe":
        assert d == r, "d and r must have the same value for GOE"
        G = np.random.randn(d, r)
        return (G+G.T)/(np.sqrt(2))

In [125]:
d = 8
r = 7
dist = "goe"
# Y = np.random.randn(d, r)
max_ = float("-inf")
for i in range(100000):
    Y = sample_proposal(d, r, dist)
    prob = eval_proposal(Y, dist)
    # if prob > 0.1:
    max_ = max(max_, prob)
print(max_)

AssertionError: d and r must have the same value for GOE

In [13]:
21.09/16

1.318125